Vytýkám před závorku: streamovací platformy používají typicky mladší lidi, rádio BY MĚLO BÝT PRO VŠECHNY, měli bychom na to při psaní pamatovat.

In [60]:
import os
import pandas as pd

In [61]:
radio = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))

In [62]:
stream = pd.read_csv(os.path.join("data","CZ_SINGLES_DIGITAL_TOP_100_tydenni.csv"))

In [63]:
stream['rok'].min()

2014

In [64]:
radio = radio[radio['rok'] >= stream['rok'].min()]

## Porovnání výskytů interpretů v top 100 rádio vs. stream od roku 2014

In [65]:
radio.groupby('interpret_hlavni').size().nlargest(10)

interpret_hlavni
Mirai              1180
Kryštof             930
Chinaski            880
Marek Ztracený      828
Slza                755
Ed Sheeran          717
Xindl X             705
Imagine Dragons     660
David Guetta        523
Robin Schulz        499
dtype: int64

In [66]:
stream.groupby('interpret_hlavni').size().nlargest(10)

interpret_hlavni
Viktor Sheen       1753
Imagine Dragons    1488
Ed Sheeran         1312
Calin              1072
Yzomandias          978
The Weeknd          700
Nik Tendo           695
Kygo                625
Onerepublic         559
Chainsmokers        549
dtype: int64

In [67]:
pomer = (radio.groupby('interpret_hlavni').size() / stream.groupby('interpret_hlavni').size()).sort_values()

### Co lidi streamují víc, než dramaturgové nasazují:

Metodologicky velmi problematické (poměr jednotlivých účastí jednotlivých písní v žebříčcích v rádiu / online), není to objektivní pořadí, spíš pro orientaci!

In [68]:
pomer.head(25)

interpret_hlavni
Wohnout          0.008032
Calin            0.015858
James Hype       0.016667
Matoma           0.027027
Rema             0.033333
Ben Cristovao    0.034783
Rae Sremmurd     0.040000
Kapitán Demo     0.042254
Marc E. Bassy    0.047619
Jon Bellion      0.052632
Zhu              0.058824
Labrinth         0.060000
Loud Luxury      0.061224
Queen            0.067416
Timeflies        0.078947
Surfaces         0.080000
Doja Cat         0.083916
Cardi B          0.085366
Skrillex         0.085366
Rixton           0.086957
Cheat Codes      0.087912
Luude            0.088235
Drake            0.089506
Fetty Wap        0.095238
Ego              0.100000
dtype: float64

### Co dramaturgové nasazují víc, než lidi poslouchají:

In [69]:
pomer.dropna().nlargest(25)

interpret_hlavni
Poetika                   160.000000
Purple Disco Machine       74.000000
Lake Malawi                74.000000
Voxel                      68.000000
Imany                      65.000000
David Stypka               63.000000
Robbie Williams            58.000000
U2                         57.000000
Avril Lavigne              44.500000
Bakermat                   43.000000
Miro Žbirka                42.833333
Lucie                      41.500000
Biffy Clyro                41.000000
Strumbellas                29.000000
Junge Junge                26.750000
Marek Ztracený             26.709677
Ondřej Brzobohatý          23.000000
Martin Solveig             22.666667
Nico Santos                22.200000
R3Hab                      21.666667
James Blunt                20.250000
Becky Hill                 20.200000
Mans Zelmerlöw             18.666667
Jelen                      17.321429
Thirty Seconds To Mars     16.400000
dtype: float64

ZDŮRAZŇUJI: jde o prezenci v top 100, ne poměr real vysílání / streamů. Může být zkreslené, nejde o objektivní pořadí, ale dost to napovídá.

### Rodíl nasazovanosti x streamovanosti žánrů

In [70]:
wikid = pd.read_json(os.path.join("wikidata","wikidata.json"))

In [71]:
zanry = pd.DataFrame(pomer).rename(columns={0:'nasazovanost'}).merge(wikid[['interpret_hlavni','zanry']], on='interpret_hlavni')

In [72]:
zanry.explode('zanry').groupby('zanry')['nasazovanost'].mean().dropna().nsmallest(30)

zanry
klasický heavy metal         0.067416
drill                        0.089506
G-Funk                       0.147727
West Coast rap               0.147727
conscious hip hop            0.147727
jazz rap                     0.147727
political hip hop            0.147727
Horrorcore                   0.158038
Midwest hip hop              0.158038
dirty rap                    0.158038
East Coast hip hop           0.186047
big band                     0.195652
swing                        0.195652
cloud rap                    0.220503
Country rap                  0.228495
hardcore hip hop             0.237239
samba                        0.262115
Gangsta rap                  0.263639
comedy hip hop               0.280826
SoundCloud rap               0.308511
lo-fi music                  0.308511
future garage                0.384615
leftfield house              0.384615
Chillwave                    0.419355
bluegrass                    0.419355
garage rock                  0.419355
hudebn

In [73]:
zanry.explode('zanry').groupby('zanry')['nasazovanost'].mean().dropna().nlargest(30)

zanry
jazz rock                 63.000000
post-punk                 57.000000
post-grunge               44.500000
dance-rock                29.020000
soft rock                 23.143243
Britpop                   21.437097
blues                     17.861671
American folk music       17.321429
punk rock                 15.631729
pop soul                  13.600000
lidová hudba              13.470255
hardstyle                 11.612745
alternativní rock          9.398100
folk                       9.371583
akustická hudba            9.285714
jazz house                 9.000000
indie rock                 8.404081
progresivní rock           8.233708
dark wave                  8.000000
industriální hudba         8.000000
industriální rock          8.000000
pop rock                   7.851114
indie folk                 7.605053
latinskoamerická hudba     7.123077
big room house             6.822917
taneční hudba              6.350383
indie pop                  5.787820
tradiční pop          

Zase: metodologicky je to ošementý, ale obrázek to dělá moc hezkej.

### Co lidi streamují, ale od roku 2014 nebylo v rádových TOP 100:

In [74]:
streamovani = stream['interpret_hlavni'].drop_duplicates().to_list()
nevysilani = [s for s in streamovani if len(radio[radio['interpret_hlavni'] == s]) == 0]
stream[stream['interpret_hlavni'].isin(nevysilani)].groupby('interpret_hlavni').size().nlargest(60)

interpret_hlavni
Viktor Sheen        1753
Yzomandias           978
Nik Tendo            695
Ektor                531
Ca$Hanova Bulhar     338
Travis Scott         237
Stein27              225
Earth                198
Xxxtentacion         195
Robin Zoot           192
Neighbourhood        179
P T K                178
Škwor                158
Milion+              154
Dorian               150
Renne Dang           145
Tom Odell            140
Smack                137
Juice Wrld           129
Sergei Barracuda      93
Koky                  92
Hasan                 88
Grey256               86
Kojo                  84
Gleb                  75
Kontrafakt            74
Decky                 69
Rest                  67
Arctic Monkeys        66
Sto Zvířat            66
Sima                  65
Dababy                58
Samey                 56
Separ                 56
Kanye West            55
Hard Rico             54
Flume                 52
Kiiara                52
Rammstein             52
Elley Du

NEMŮŽU UVĚŘIT ŽE TOTO NENÍ CHYBA

## Potenciální námět na domácí úkol: vydestilovat ze všech žánrů pár základních

In [75]:
len(wikid.explode('zanry')['zanry'].drop_duplicates())

315

In [76]:
wikid.explode('zanry')['zanry'].drop_duplicates().to_list()

['Southern hip hop',
 'hip hop',
 'Gangsta rap',
 'trap',
 'hardcore hip hop',
 'Atlanta hip hop',
 'dirty south',
 'pop rap',
 'synthpop',
 'electro hop',
 'pop rock',
 'pop punk',
 'East Coast hip hop',
 None,
 'SoundCloud rap',
 'reggaeton',
 'pop music',
 'cloud rap',
 'psychedelic rap',
 'experimentální hip hop',
 'alternativní hip-hop',
 'hard rock',
 'rock and roll',
 'blues rock',
 'Europop',
 'euro disco',
 'taneční hudba',
 'glam rock',
 'speed metal',
 'power metal',
 'heavy metal',
 'dance-pop',
 'elektronická hudba',
 'soul',
 'bílý soul',
 'rhythm and blues',
 'jazz',
 'rock',
 'barokní pop',
 'alternative pop',
 'Avant-pop',
 'dark ambient',
 'indie pop',
 'chamber pop',
 'electro house',
 'progressive house',
 'Dutch house',
 'contemporary R&B',
 'dancehall',
 'reggae',
 'elektronická taneční hudba',
 'deep house',
 'alternativní rock',
 'experimentální rock',
 'post-grunge',
 'electronica',
 'progresivní rock',
 'alternativní R&B',
 'house music',
 'blues',
 'shock roc